# 栄養パターンに基づく口腔・上部消化管がん分類モデルの構築および評価４（栄養パターンVer）

In [ ]:
%reset -f

# 概要

## パッケージインストール

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.linear_model import LinearRegression
!pip install statsmodels
from statsmodels.duration.hazard_regression import PHReg
!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from decimal import Decimal, ROUND_HALF_UP
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
import time
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, make_scorer
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
import io
from contextlib import redirect_stdout
from IPython.display import display
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer 
import shap
import os

## データの読み込み

In [ ]:
# データの読み込み
adjusted_z_df=pd.read_csv("adjusted_z_df.csv")
pcaDF_TN_proj=pd.read_csv('pcaDF_TN_proj.csv')

In [ ]:
display(pcaDF_TN_proj)

## 学習用データとテスト用データの作成

In [ ]:

xDF_all = adjusted_z_df.copy()

#Prepare outputs/classes of all samples

yDF_all = pcaDF_TN_proj["TargetName"]
#Check
print('Inputs:')
display(xDF_all)
display(xDF_all.shape)
print('Outputs:')
display(yDF_all)
display(yDF_all.shape)

In [ ]:
#Split the dataset into training and testing sets
xDF_train, xDF_test, yDF_train, yDF_test = train_test_split(xDF_all, yDF_all,
                                                            test_size=0.2, shuffle=True, random_state=123)

#Check the np.array shape
display(xDF_train.shape)
display(xDF_test.shape)
display(yDF_train.shape)
display(yDF_test.shape)
display(xDF_train)
display(xDF_test)
display(yDF_train)

In [ ]:
# TargetNameのデータ数確認
yDF_test.value_counts(dropna=False)

In [ ]:
# TargetNameのデータ数確認
yDF_train.value_counts(dropna=False)

## ハイパーパラメータの最適化

In [ ]:
n_estimators_space = np.unique([int(x) for x in np.logspace(1, 3, num=7, endpoint=True)])
n_estimators_space = n_estimators_space[n_estimators_space > 0]
# Prepare hyperparameter space to be searched 
param_space = {
    'n_estimators': n_estimators_space, 
    'max_depth': [int(i) for i in range(3, 11)],  #1から10まで
    
    'min_samples_leaf': [int(i) for i in np.unique(np.logspace(0, 2, num=10, endpoint=False))], 
    # np.logspace(0, 2, num=10) で 1から100までを対数的に10点でカバー。
    
    'max_features': ['sqrt', 'log2', 4, 5, 10, 15, 21, 26, 30], 
    # 21で頭打ちの可能性を考慮し、21より大きい値を手動で追加 (例: 26, 30)。
}

#Prepare a model while setting hyperparameters
model = RandomForestClassifier(criterion='gini',
                               #max_depth=None,
                               min_samples_split=2,
                               #min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               #max_features=None,
                               random_state=123,#Specified for reproducibility
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               class_weight='balanced',
                               ccp_alpha=0.0)

#Prepare CV generator
cv_gen = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)#Specified for reproducibility

#Wrap the model for tuning hyperparameters with cross-validation
modelcv = GridSearchCV(model, param_grid=param_space,
                       scoring=None, n_jobs=None, refit=True, cv=cv_gen,
                       verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

#Fit models with all set of parameters on the training set
modelcv.fit(xDF_train, yDF_train.values.ravel())

#Check the CV results
print('Best score:', modelcv.best_score_)
print('Best hyperparameters:', modelcv.best_params_)
tempDF = pd.DataFrame(modelcv.cv_results_).sort_values('mean_test_score', ascending=False)
display(tempDF)

In [ ]:
# 最適なモデルを取得
best_model = modelcv.best_estimator_

# モデルをファイルに保存
filename = 'best_random_forest_None_model.joblib'
joblib.dump(best_model, filename)

print(f"最適なモデルを {filename} に保存しました。")

In [ ]:
filename = 'best_random_forest_None_model.joblib'
model = joblib.load(filename)

## 分類モデルの精度分析

### 分類モデルの精度結果

In [ ]:
#Evaluate the model on the testing set, based on classification accuracy
score = model.score(xDF_test, yDF_test)
print('Classification accuracy:', score)

#Predict classes of the testing set
tempA = model.predict(xDF_test)
##Clean
tempS = pd.Series(tempA, index=xDF_test.index, name='PredictedClass')
tempDF = yDF_test.rename('TrueClass') 
tempDF = pd.merge(tempS, tempDF, left_index=True, right_index=True, how='left')

#Check
#display(tempDF)
#display(tempDF.describe(include='all'))
display(tempDF['TrueClass'].value_counts())
display(tempDF['PredictedClass'].value_counts())
display(tempDF.loc[tempDF['TrueClass']!=tempDF['PredictedClass']])

### 決定木モデルの一例

In [ ]:
# 1. 学習済みのRandomForestClassifierから、構成要素である単一の決定木を取り出す
#    ここでは、model.estimators_[0]（0番目の木）を使用
sns.set(style='ticks', context='talk')
single_tree = model.estimators_[0]

# 2. 可視化の実行
plt.figure(figsize=(40, 30))
plot_tree(
    decision_tree=single_tree,                     
    feature_names=xDF_train.columns.tolist(),      
    class_names=model.classes_.astype(str).tolist(),
    filled=True, 
    node_ids=True,
    fontsize=20
)

plt.show()

### 特徴量の重要度

In [ ]:
importance_scores = model.feature_importances_
feature_names = xDF_train.columns.tolist()
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance_scores
})
sorted_importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(sorted_importance_df)

### 混同行列

In [ ]:
y_pred = model.predict(xDF_test)
print(classification_report(yDF_test.values.ravel(), y_pred))

In [ ]:
y_pred = model.predict(xDF_test)
tempDF_true = yDF_test.rename('TrueClass') 

# PredictedClass (予測) の準備: Seriesを作成
tempS_pred = pd.Series(y_pred, index=xDF_test.index, name='PredictedClass')

# 結合して最終的な評価用DataFrame (tempDF) を作成
tempDF = pd.merge(tempS_pred, tempDF_true, left_index=True, right_index=True, how='left')
TEXT_OUTPUT_FILENAME = 'NUT_model_evaluation_report.txt'
CLASSIFICATION_REPORT_DF_FILENAME = 'NUT_classification_report_df.csv' # または .xlsx

# バッファを用意し、print出力をリダイレクト
f = io.StringIO()
with redirect_stdout(f):
    print('--- モデル評価結果 ---')
    print()
    # Accuracy
    accuracy = accuracy_score(tempDF['TrueClass'], tempDF['PredictedClass'], normalize=True)
    print('Accuracy:', accuracy)
    # Cf. Manual calculation (手動計算)
    manual_accuracy = (tempDF['TrueClass']==tempDF['PredictedClass']).sum()/len(tempDF)
    print('-> Cf. Manual calculation:', manual_accuracy)

    print('\n' + '='*30 + '\n')

    ## 混同行列の表示 (ここでは表示はスキップし、画像で保存する)
    print('## 混同行列の表示')
    print('Confusion Matrix (グラフ) は ' + 'NUT_confusion_matrix.png' + ' に保存されます。')

    print('\n' + '='*30 + '\n')

    ## Precision, Recall, F1-score の DataFrame 出力
    # output_dict=True で結果を辞書として取得
    tempD = classification_report(tempDF['TrueClass'], tempDF['PredictedClass'], output_dict=True)

    # 辞書をDataFrameに変換
    report_df = pd.DataFrame(tempD).T

    # 最終レポートDataFrameをCSVとして保存
    report_df.to_csv(CLASSIFICATION_REPORT_DF_FILENAME, float_format='%.4f')
    print(f'Classification Report (DataFrame T) は {CLASSIFICATION_REPORT_DF_FILENAME} にCSVとして保存されました。')
    print('\nClassification Report (DataFrame T) の内容:')
    # print関数でDataFrameの内容をファイルに書き込み
    print(report_df.to_string()) # to_string()で整形されたテキスト形式に変換して出力

# リダイレクトされた出力をファイルに書き込み
with open(TEXT_OUTPUT_FILENAME, 'w', encoding='utf-8') as text_file:
    text_file.write(f.getvalue())

print(f'\n✅ 全てのテキスト出力は "{TEXT_OUTPUT_FILENAME}" に保存されました。')
print(f'✅ Classification Report のDataFrameは "{CLASSIFICATION_REPORT_DF_FILENAME}" にCSVとして保存されました。')
CONFUSION_MATRIX_FILENAME = 'NUT_confusion_matrix.png'

# Visualize confusion matrix
ConfusionMatrixDisplay.from_predictions(tempDF['TrueClass'], tempDF['PredictedClass'])
plt.title('Confusion Matrix') # タイトルを追加
# plt.show() の代わりに plt.savefig() でファイルに保存
plt.savefig(CONFUSION_MATRIX_FILENAME, bbox_inches='tight')
plt.close() # グラフ表示を抑止するために閉じる（環境によっては必須）

print(f'✅ 混同行列のグラフは "{CONFUSION_MATRIX_FILENAME}" にPNG画像として保存されました。')

print('\n\n--- 実行結果の確認 (画面表示) ---')
print('Accuracy:', accuracy)
print('-> Cf. Manual calculation:', manual_accuracy)
print('\n' + '='*30 + '\n')
print('Classification Report (DataFrame T):')
display(report_df)
ConfusionMatrixDisplay.from_predictions(tempDF['TrueClass'], tempDF['PredictedClass'])
plt.title('Confusion Matrix')
plt.show()

### ROC曲線

In [ ]:
y_score = model.predict_proba(xDF_test)
ptnA={'non_cancer':0,'oral_cancer':1}
yDF_test_1=yDF_test.map(ptnA)
y_true_binary = yDF_test_1.values.ravel() 

fpr, tpr, thresholds = roc_curve(y_true_binary, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 5))

# ROC曲線のプロット
plt.plot(
    fpr, tpr, color='darkorange', lw=2, 
    label=f'ROC curve (area = {roc_auc:0.2f})'
)

# ランダムな分類器の対角線 (AUC=0.5)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# グラフの設定
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)') # 偽陽性率
plt.ylabel('True Positive Rate (TPR) / Recall') # 真陽性率 / 再現率
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f'\nAUC (Area Under the Curve): {roc_auc:.4f}')
print('\n' + '='*30 + '\n')

### SHAP値の出力

In [ ]:
shap.initjs()

# 1. SHAP Explainer の作成 (modelとxDF_trainは定義済みと仮定)
explainer = shap.TreeExplainer(model)

# 2. SHAP値の計算 (実行済み)
shap_values = explainer.shap_values(xDF_train) 

# 適切なSHAP値の抽出 (クラス1のSHAP値)
shap_values_class1 = shap_values[:, :, 1]


# 3. 特徴量のグローバルな重要度 (Mean Absolute SHAP Value) の計算 
shap_v = np.abs(shap_values_class1).mean(axis=0)


# 4. DataFrameの作成 (特徴量名と重要度を結合)
feature_names = xDF_train.columns.tolist()

shap_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP_Importance': shap_v
})


# 5. 重要度が高い順にソートして表示 
sorted_shap_importance_df = shap_importance_df.sort_values(by='SHAP_Importance', ascending=False)
print("--- SHAP Based Feature Importance (Mean Absolute SHAP Value) ---")
print(sorted_shap_importance_df)


# 6. SHAP Summary Plotの表示 
print("\n--- SHAP Summary Plot ---")
# Summary Plotに (サンプル数, 特徴量数) の配列を渡す
shap.summary_plot(shap_values_class1, xDF_train)

In [ ]:
# Plot保存のために追加
shap.initjs()
# 1. SHAP Explainer の作成 
explainer = shap.TreeExplainer(model)
# 2. SHAP値の計算 (実行済み)
shap_values = explainer.shap_values(xDF_train) 
# 適切なSHAP値の抽出 (クラス1のSHAP値)
shap_values_class1 = shap_values[:, :, 1]


# 3. 特徴量のグローバルな重要度 (Mean Absolute SHAP Value) の計算
shap_v = np.abs(shap_values_class1).mean(axis=0)


# 4. DataFrameの作成 
feature_names = xDF_train.columns.tolist()
shap_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP_Importance': shap_v
})

# 5. 重要度が高い順にソートして表示
sorted_shap_importance_df = shap_importance_df.sort_values(by='SHAP_Importance', ascending=False)
print("--- SHAP Based Feature Importance (Mean Absolute SHAP Value) ---")
print(sorted_shap_importance_df)

csv_filename = 'NUT_shap_feature_importance.csv'
sorted_shap_importance_df.to_csv(csv_filename, index=False)
print(f"\n✅ 特徴量の重要度データを '{csv_filename}' に保存しました。")

# 6. SHAP Summary Plotの表示と保存
print("\n--- SHAP Summary Plotの保存 ---")
# show=False を設定して、プロットの表示を抑制し、plt.savefig() で保存できるようにする
shap.summary_plot(shap_values_class1, xDF_train, show=False)

# プロットをファイルとして保存
plot_filename = 'NUT_shap_summary_plot_class1.png'
plt.savefig(plot_filename, bbox_inches='tight') 
plt.close() # Figureを閉じる

print(f"\n✅ SHAP Summary Plotを '{plot_filename}' に保存しました。")

In [ ]:
try:
    base_value_class1 = explainer.expected_value[1]
except Exception as e:
    # 予期せぬエラーが発生した場合の代替処理 (デバッグ用)
    print(f"Error accessing explainer.expected_value[1]: {e}")
    print("explainer.expected_valueの内容を確認してください。")
    # 代替値（実行を継続するための一時的な措置）
    # 通常はこの処理は不要ですが、エラーを防ぐために配置
    base_value_class1 = 0.0 # 実際の値に合わせて修正が必要です

print(f"Base Value for Class 1 (Expected Output): {base_value_class1:.4f}")

# 3. Force Plotの表示 (NameErrorが発生した部分)
print("\n--- SHAP Force Plot for Multiple Samples (Class 1) ---")
shap.force_plot(
    base_value_class1,       # ここで定義した変数を使用
    shap_values_class1[:100, :],
    xDF_train.iloc[:100, :],
    matplotlib=False
)

In [ ]:
feature_names = xDF_train.columns.tolist()

print("--- SHAP Dependence Plots for ALL Features (Class 1) ---")

# 全ての特徴量についてループ処理を実行
for feature in feature_names:
    print(f"\n[Plotting Dependence Plot for: {feature}]")
    shap.dependence_plot(
        feature,             
        shap_values_class1,  
        xDF_train            
    )

In [ ]:
#最も重要度の高い特徴量と、次に高い特徴量の相互作用を見る
target_feature_1 = sorted_shap_importance_df['Feature'].iloc[0]
interaction_feature_2 = sorted_shap_importance_df['Feature'].iloc[1]

print(f"\n--- SHAP Dependence Plot with Interaction: {target_feature_1} vs {interaction_feature_2} ---")

shap.dependence_plot(
    target_feature_1,           # X軸: プロットしたい特徴量
    shap_values_class1,
    xDF_train,
    interaction_index=interaction_feature_2 # 色付けに使う特徴量
)

### ROC曲線の合併

In [ ]:
filename = 'best_random_forest_None_c_model.joblib'
# joblib.load()を使用してモデルをメモリに読み込み、変数 'model' に格納
model1 = joblib.load(filename)

In [ ]:
xDF_test1 = pd.read_csv('xDF_test.csv')
yDF_test1 = pd.read_csv('yDF_test.csv').iloc[:, 0]
print(f"xDF_test1 の型: {type(xDF_test1)}")
print(f"yDF_test1 の型: {type(yDF_test1)}")

In [ ]:
# 1. 予測確率の取得
y_score = model1.predict_proba(xDF_test1)

# 2. 正解ラベルの準備 (二値化)
ptnA={'non_cancer':0,'oral_cancer':1}
yDF_test1=yDF_test1.map(ptnA)
y_true_binary = yDF_test1.values.ravel() 

# 3. ROC曲線の計算
fpr1, tpr1, thresholds = roc_curve(y_true_binary, y_score[:, 1])
roc_auc1 = auc(fpr1, tpr1)

# 4. ROC曲線のプロット
plt.figure(figsize=(6, 5))

# ROC曲線のプロット
plt.plot(
    fpr1, tpr1, color='darkorange', lw=2, 
    label=f'ROC curve (area = {roc_auc1:0.2f})'
)

# ランダムな分類器の対角線 (AUC=0.5)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# グラフの設定
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)') # 偽陽性率
plt.ylabel('True Positive Rate (TPR) / Recall') # 真陽性率 / 再現率
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f'\nAUC (Area Under the Curve): {roc_auc1:.4f}')

In [ ]:
plt.figure(figsize=(7, 6)) # グラフの領域を一度だけ作成
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random (AUC = 0.50)')

# ROC曲線のプロット (Model)
plt.plot(
    fpr, tpr, color='darkorange', lw=2,
    label=f'nut ROC curve (area = {roc_auc:.2f})'
)
# ROC曲線のプロット (Model1)
plt.plot( # 続けてプロットすることで、同じグラフに線が追加される
    fpr1, tpr1, color='green', lw=2, linestyle='-',
    label=f'PC ROC curve (area = {roc_auc1:.2f})'
)

plt.legend(loc="lower right", fontsize=12) 
# その他のグラフ設定
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR) or Recall')
plt.title('Receiver Operating Characteristic (ROC) Curve Comparison')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(7, 6)) # グラフの領域を一度だけ作成
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random (AUC = 0.50)')

# ROC曲線のプロット (Model)
plt.plot(
    fpr, tpr, color='darkorange', lw=2,
    label=f'nut ROC curve (area = {roc_auc:.2f})'
)
# ROC曲線のプロット (Model1)
plt.plot(
    fpr1, tpr1, color='green', lw=2, linestyle='-',
    label=f'PC ROC curve (area = {roc_auc1:.2f})'
)

plt.legend(loc="lower right", fontsize=12) 

# その他のグラフ設定
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR) or Recall')
plt.title('Receiver Operating Characteristic (ROC) Curve Comparison')
plt.grid(True)

# プロットをファイルとして保存
plot_filename = 'roc_curve_comparison.png'
plt.savefig(plot_filename, bbox_inches='tight')

plt.close()

print(f"✅ ROC曲線比較プロットを '{plot_filename}' に保存しました。")